# Logistic Regression Real Life problem


## Binary Customer Churn


## Solution 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('problemSolution').getOrCreate()


In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [9]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
final_data = output.select('features','churn')

In [15]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [17]:
from pyspark.ml.classification import LogisticRegression 

In [19]:
lr_churn = LogisticRegression(labelCol='churn')

In [22]:
fitted_churn_model = lr_churn.fit(train_churn)

In [23]:
training_sum = fitted_churn_model.summary

In [24]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                648|                648|
|   mean|0.16512345679012347|0.11574074074074074|
| stddev| 0.3715787574503436| 0.3201609053643437|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evalute Result

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [27]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.50554799247887...|[0.98907317986119...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.58764971323549...|[0.99949358610837...|       0.0|
|[29.0,11274.46,1....|    0|[4.32351101996469...|[0.98692008240435...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.20034445652417...|[0.96084723765413...|       0.0|
|[31.0,5387.75,0.0...|    0|[2.61448154305528...|[0.93178779181219...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.47070908672576...|[0.99845426540745...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.25359202169505...|[0.98598609252135...|       0.0|
|[31.0,11297.57,1....|    1|[0.95777330876525...|[0.72267576302250...|       0.0|
|[31.0,12264.68,1....|    0|[3.37282190521356...|[0.96684427062735...|       0.0|
|[32.0,5756.12,0

In [30]:
#using auc
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [31]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [32]:
auc

0.7900300433960165

### Predict on brand new unlabeled data
We still need to evaluate the new_customers.csv file!